In [ ]:
import math
import pandas as pd
import numpy as np
import scipy
import random
import math 
from scipy import stats
from csv import reader
random.seed(120)

In [ ]:
path1 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/historical parameters.csv"
path2 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/rates_instrument.csv"
path3 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/rates_exchange.csv"
path4 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/rates_interest.csv"
path5 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/tenor.csv"
path6 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/transaction_bond_hist_d.csv"
path7 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/spread_rate.csv"
path8 = "/mnt/c/Users/preety.tiwari/Documents/Market_Risk_modelling/Bond_Datasets/rates_interest_hp.csv"

In [ ]:
def getData(path, fileext = None):
    if fileext == ".csv":
        df = pd.read_csv(path)
    elif fileext == ".xlsx" or fileext == ".xls":
        df = pd.read_excel(path)
    else:
        df = pd.read_csv(path)
    return df

In [ ]:
hp = getData(path1)
n = hp['no_of_days']
n = list(n)
n = n[0]
decay_factor = hp['decay_factor']
holding_period = hp['holding_period']
confidence_level = hp['confidence_level']
decay_factor = list(decay_factor)
decay_factor = decay_factor[0]

In [ ]:
hp

# Rates Instrument

In [ ]:
##Importing dataset rates_instrument
rates_instrument= getData(path2)

In [ ]:
#Function to get a specific type of data from the particular column in data(Filtering)
def loadData(df, colname, types = None):
    
    res_df = df[df[colname] == types].reset_index(drop=True)
    return res_df
    

In [ ]:
rates_instrument_D = loadData(rates_instrument,'instrument_type',types = 'D')
rates_instrument_E = loadData(rates_instrument,'instrument_type',types = 'E')

In [ ]:
rates_instrument_D

# Rates Exchange

In [ ]:
rates_exchange = getData(path3) 

In [ ]:
rates_exchange

In [ ]:
#Generating key
rates_exchange["key"] = rates_exchange["currency_from"] + '_' + rates_exchange["currency_to"]
rates_exchange

# Rate Interest

In [ ]:
rates_interest = getData(path4)

In [ ]:
#Generating key between curve and tenor
rates_interest["key"] = rates_interest["curve"] + '_' + rates_interest["tenor"]
rates_interest.head()

In [ ]:
##Adding current rate to the top of the rates
def sim_returns(df):
    simple_returns = pd.DataFrame([])

    for a, g in df.groupby('key'):
        
        g = g.reset_index(drop = True)
        g['simple_returns'] = g['rate']-g['rate'].shift(-1)
        #new_row = g.iloc[0].to_dict()
        #new_row['simple_returns'] = [0]
        #top_row = pd.DataFrame(new_row)
        g['simulated rates'] =( g['simple_returns'] + g['rate'].iloc[0]).shift(fill_value = g['rate'][0])
        #g = g.iloc[:-1 , :]
        #g = pd.concat([top_row, g]).reset_index(drop = True)

        simple_returns = simple_returns.append(g)
        
    
    return simple_returns

In [ ]:
simulated_interest = sim_returns(rates_interest)
simulated_interest.head()

In [ ]:
simulated_interest

In [ ]:
simulated_interest.iloc[250]

# Tenors

In [ ]:
tenors = getData(path5)


# Transaction file

In [ ]:
tx_bond = getData(path6)
tx_bond

In [ ]:
#print(tx_bond["instrument"],_ ,tx_bond["k"])
tx_bond["key"] = tx_bond['instrument'].map(str) + '_' + tx_bond['k'].map(str) 
tx_bond

In [ ]:
##Sorting data by'key' column
tx_bond= tx_bond.sort_values(by=['key']).reset_index(drop=True)
tx_bond

In [ ]:
#Converting to correct datetime format
tx_bond["c_date"] = pd.to_datetime(tx_bond["current_date"],format =  "%d-%m-%Y")
tx_bond.head(10)

In [ ]:
#Adding c_date and residual_maturity_expected column
tx_bond['m_date_E'] = tx_bond['c_date'] + pd.to_timedelta(tx_bond['residual_maturity_expected'], unit='d')
tx_bond.head(10)

In [ ]:
#Adding c_date and residual_maturity_Actual column
tx_bond['m_date_A'] = tx_bond['c_date'] + pd.to_timedelta(tx_bond['residual_maturity_Actual'], unit='d')
tx_bond

In [ ]:
tx_bond["lower_tenor_key"] = tx_bond["curve"]+ "_" + tx_bond["lower_tenor"]
tx_bond.head()

In [ ]:
tx_bond["upper_tenor_key"] = tx_bond["curve"]+ "_" + tx_bond["upper_tenor"]
tx_bond

In [ ]:
lower_tenor_key_list = list(tx_bond["lower_tenor_key"])
upper_tenor_key_list = list(tx_bond["upper_tenor_key"])
tenor_list = lower_tenor_key_list + upper_tenor_key_list
tenor_list = set(tenor_list)
tenor_list = list(tenor_list)
tenor_df = pd.DataFrame(tenor_list).reset_index(drop = True)
tenor_df = tenor_df.rename(columns = {0:'tenor_list'})

In [ ]:
tenor_df

In [ ]:
#tenor_list_unique = tenor_list_unique.rename(columns = {0:'tenor_list_unique'})
tenor_df = tenor_df.sort_values('tenor_list')
tenor_df = tenor_df.reset_index(drop=True)

Matching the key values of simulated_interest dataframe with the tenor_list_unique key values of tenor_list_unique dataframe 
and dropped out the columns 'key','date','rate_stressed','simple_returns','simulated rates'.

In [ ]:
rates_interest_current = pd.merge(tenor_df,simulated_interest,how='left',left_on='tenor_list',right_on='key').drop(['key','date','rate_stressed','simple_returns','rate'], axis=1)

In [ ]:
rates_interest_current.head(50)

In [ ]:
from copy import deepcopy

In [ ]:
test_rate_df = deepcopy(rates_interest_current)
test_rate_df


In [ ]:
rates_interest_current_1V = (test_rate_df.drop_duplicates(['tenor_list']))
rates_interest_current_1V = rates_interest_current_1V.reset_index(drop=True)
rates_interest_current_1V

In [ ]:
###Filtering according to Instrument
rates_interest_current_1S = rates_interest_current.filter(['tenor_list', 'simulated rates']) ##To extract particular columns
rates_interest_current_1S

In [ ]:
rates_interest_current_1S = rates_interest_current_1S.set_index('tenor_list')
rates_interest_current_1S

In [ ]:
##Fetching current values for each of the tenor_list_unique

In [ ]:
##Segregating simulations according to the tenor_list_unique(unique key) generated
df = (rates_interest_current_1S.assign(labels = rates_interest_current_1S.groupby(level = 0).cumcount())
            .groupby([rates_interest_current_1S.index,'labels']).first()
            .unstack('labels')
            .sort_index(axis =1,level = 1)
            .droplevel(1,axis = 1))
df

In [ ]:
#Extracting values at 0th index
tenor_rates = df.iloc[:,0]
tenor_rates

In [ ]:
tenor_rates_df_V = pd.DataFrame(tenor_rates)

In [ ]:
tenor_rates_df_V

In [ ]:
tenor_rates_df_V.index.rename('key',inplace=True)

In [ ]:
tenor_rates_df_V

Match lower tenor key with the tenor_rate df and get the corresponding rate values.
Match upper tenor key with the tenor_rate df and get the corresponding rate values


In [ ]:
tx_bond_new = tx_bond.merge(tenor_rates_df_V,how='inner',left_on='lower_tenor_key',right_on='key')

In [ ]:
tx_bond_new

In [ ]:
##Renaming the rate column
tx_bond_new = tx_bond_new.rename(columns = {'simulated rates':'lower_rates'})
tx_bond_new

In [ ]:
#Upper rates
tx_bond_new = tx_bond_new.merge(tenor_rates_df_V, how='inner', left_on='upper_tenor_key', right_on='key')

In [ ]:
tx_bond_new

In [ ]:

tx_bond_new =  tx_bond_new.rename(columns = {'simulated rates':'upper_rates'})

In [ ]:
tx_bond_new

In [ ]:
#Sorting values by lower-rates
tx_bond_new = tx_bond_new.sort_values(by=['lower_rates'])


In [ ]:
tx_bond_new

In [ ]:
# Interpolated rates

In [ ]:
def interpolate_values(df):

    numerator1 = (df['upper_rates'] - df['lower_rates'])
    numerator2 = (df['residual_maturity_expected'] - df['lower_maturity'])
    numerator = numerator1*numerator2
    denominator = (df['upper_maturity'] - df['lower_maturity'])
    interpolate_col = df['lower_rates'] + (numerator/denominator)
    return interpolate_col


In [ ]:
tx_bond_new['Interpolated_rates'] = tx_bond_new.apply(lambda x: x['upper_rates'] if x['upper_tenor_key'] == x['lower_tenor_key'] else interpolate_values(x), axis=1)

In [ ]:
tx_bond_new

### SPREAD's

In [ ]:
spreads= getData(path7)
spreads

In [ ]:
##Dropping duplicates
spread_rates = (spreads.drop_duplicates(['instrument']))
spread_rates.columns= spread_rates.columns.str.lower() ##Column names to lower case
spread_rates.head()

In [ ]:
##Renaming rates to spread_rates
spread_rates =  spread_rates.rename(columns = {'rate':'spread_rates'})
spread_rates.head()

In [ ]:
s_rate_df = spread_rates[['instrument', 'spread_rates']].reset_index(drop=True)

In [ ]:
s_rate_df

In [ ]:
tx_bond_revamp = tx_bond_new.merge(s_rate_df, on ='instrument', how='left')

In [ ]:
tx_bond_revamp

In [ ]:
#Replacing NAN's with 0's
tx_bond_revamp['spread_rates'] = tx_bond_revamp['spread_rates'].fillna(0)
tx_bond_revamp

In [ ]:
#YTM(Yield to Maturity)
tx_bond_revamp['YTM']= tx_bond_revamp['Interpolated_rates'] + tx_bond_revamp['spread_rates'] + tx_bond_revamp['mark_up_value'] 

In [ ]:
tx_bond_revamp

In [ ]:
#result = tx_bond_revamp.dtypes
#result

In [ ]:
tx_bond_revamp['maturity_period_E'] = (tx_bond_revamp['m_date_E'] - tx_bond_revamp['c_date']).dt.days
tx_bond_revamp

In [ ]:
tx_bond_revamp['maturity_period_A'] = (tx_bond_revamp['m_date_A'] - tx_bond_revamp['c_date']).dt.days
tx_bond_revamp

In [ ]:
##Condition

In [ ]:
tx_bond_revamp['no_of_days_in_a_year'] = ''
for i in range (len(tx_bond_revamp)):

  
  if  tx_bond_revamp['intbasis'][i] == 0:
        tx_bond_revamp['no_of_days_in_a_year'][i] = 360
  
  elif tx_bond_revamp['intbasis'][i] == 1: 
        tx_bond_revamp['no_of_days_in_a_year'][i] = 365  
  
  elif tx_bond_revamp['intbasis'][i] == 2: 
        tx_bond_revamp['no_of_days_in_a_year'][i] = 360 
  
  elif tx_bond_revamp['intbasis'][i] == 3: 
        tx_bond_revamp['no_of_days_in_a_year'][i] = 365 
  
  elif tx_bond_revamp['intbasis'][i] == 4: 
        tx_bond_revamp['no_of_days_in_a_year'][i] = 360
   
  else:
        pass
  

In [ ]:
tx_bond_revamp

In [ ]:
#cashflow_mtm
tx_bond_revamp['cashflow_mtm'] = tx_bond_revamp['Cashflow'] / ((1+(tx_bond_revamp['YTM']/100))**(tx_bond_revamp['residual_maturity_Actual']/tx_bond_revamp['no_of_days_in_a_year'])) 

In [ ]:
tx_bond_revamp

In [ ]:
##Sorting data by'key' column
tx_bond_revamp= tx_bond_revamp.sort_values(by=['key']).reset_index(drop=True)
tx_bond_revamp

In [ ]:
tx_bond_revamp

In [ ]:
##MTM

In [ ]:
df2 = tx_bond_revamp.groupby('id')['cashflow_mtm'].sum()
mtm = list(df2)
mtm = [round(item, 0) for item in mtm]
mtm = pd.DataFrame(mtm)
mtm

In [ ]:
def get_mtm_bc(tx_eq):
    
    res_df = list()
    for k, h in tx_eq.groupby('id'):
        result = {'idx':h["id"].iloc[0] , "env":k,"instrument":h["instrument"].iloc[0] ,"currency":h["currency"].iloc[0] ,"currency_base":h["currency_base"].iloc[0], "face_value":h["Face_value"].iloc[0], "maturity_amount":h["Maturity_Amount"].iloc[0]}
        res_df.append(result)
        #res_df = pd.DataFrame(res_df)

    return res_df

In [ ]:
#, "bondwise_current_mtm":mtm[0]
red = get_mtm_bc(tx_bond_revamp)
res = pd.DataFrame(red)
res

In [ ]:
#renaming the column for mtm
mtm.rename(columns = {0:'bondwise_current_mtm'}, inplace = True)
mtm = mtm.reset_index(drop = True)
mtm

In [ ]:
#concatenating res dataframe and mtm dataframe 
result_new = pd.concat([res, mtm], axis = 1)
result_new

In [ ]:
#Calibrating price
result_new['price'] = (result_new['bondwise_current_mtm']*result_new['face_value'])/result_new['maturity_amount']
result_new

### All MTMS

In [ ]:
interpolated_curve_rate = np.empty((n+1,1))

In [ ]:
lW_rate1 = np.empty((n+1,3))
up_rate1 = np.empty((n+1,3))

In [ ]:
interpolated_curve_rate = pd.DataFrame()
lw_rate1 = pd.DataFrame()
up_rate1 = pd.DataFrame()
for i in range(0,len(tx_bond_revamp)):
    
    a = tx_bond_revamp.loc[i,['lower_tenor_key', 'upper_tenor_key']]
    a_a = list(a)
    rates = simulated_interest[simulated_interest['key'].isin(a)]
    rates = rates.loc[:, ['key','simulated rates']]
    rates = rates.set_index('key')
        ##Segregating simulations according to the instrument
    rates2 = (rates.assign(labels = rates.groupby(level = 0).cumcount())
            .groupby([rates.index,'labels']).first()
            .unstack('labels')
            .sort_index(axis =1,level = 1)
            .droplevel(1,axis = 1))

    rates2 = rates2.transpose()    
    rates2 = rates2.reset_index(drop = True)
    rates2 = rates2.loc[:, [a_a[0],a_a[1]]]
    #print(rates2)
    #rates2 =rates2[,match(a,colnames(rates2))]
    rates2['key'] = tx_bond_revamp['key'][i]
    rates2['maturity']= tx_bond_revamp['residual_maturity_expected'][i]
    rates2['lower_tenor'] = tx_bond_revamp['lower_maturity'][i]
    rates2['upper_tenor'] = tx_bond_revamp['upper_maturity'][i]
    if (rates2.iat[0,4]==rates2.iat[0,5]):
        rates2['intepolated'] = rates2.iloc[:,0]

    else:
        rates2['intepolated'] = rates2.iloc[:,0]+((rates2.iloc[:,1]-rates2.iloc[:,0])*(rates2.iloc[:,3]-rates2.iloc[:,4]))/(rates2.iloc[:,5]-rates2.iloc[:,4])

    rates3=pd.DataFrame(rates2.iloc[:,-1])
    rates4=pd.DataFrame(rates2.iloc[:,0])
    rates5=pd.DataFrame(rates2.iloc[:,1])
    id=tx_bond_revamp['key'][i]

    rates3.columns = [rates2['key'][0]]
    rates4.columns = [id]
    rates5.columns = [id]
    interpolated_curve= pd.DataFrame(rates3)
    interpolated_curve_rate = pd.concat([interpolated_curve_rate,interpolated_curve], axis = 1)
    lw_rate1 = pd.concat([lw_rate1,rates4], axis = 1)
    up_rate1 = pd.concat([up_rate1,rates5], axis = 1)

In [ ]:
spreads

In [ ]:
##Unique instrument of tx_bond_revamp
aaa = tx_bond_revamp['instrument'].unique()
aaa = pd.DataFrame(aaa)
aaa


In [ ]:
##Unique instrument of spreads
zzz = spreads['instrument'].unique()
zzz = pd.DataFrame(zzz)
zzz

In [ ]:
##Finding common values between two dataframe
common = aaa.merge(zzz, on=[0])
common

In [ ]:
common = list(common[0].values)
common

In [ ]:
#Matching the spreads and tx_bond dataframes on the basis of column 'instrument'
spreads1 = spreads.loc[spreads['instrument'].isin(common)]
spreads1

In [ ]:
###True code

In [ ]:
#instru_count = pd.DataFrame()
if len(spreads1) == 0:
    names = list(interpolated_curve_rate.columns)
    spreads_unstk = pd.DataFrame(columns = names)   
else:
    spread_1 = spreads1.loc[:, ['instrument','rate']]
    spread_1 = spread_1.set_index('instrument')
    spreads_unstk = (spread_1.assign(labels = spread_1.groupby(level = 0).cumcount())
                    .groupby([spread_1.index,'labels']).first()
                    .unstack('labels')
                    .sort_index(axis =1,level = 1)
                    .droplevel(1,axis = 1))
    spreads_unstk = spreads_unstk.transpose()
    spreads_unstk = spreads_unstk.iloc[:,[1,0]]
    instru_count = tx_bond_revamp.groupby('instrument').count()
    instru_count = pd.DataFrame(instru_count['k'])
    instru_count['instrument'] = instru_count.index
    instru_count = instru_count.reset_index(drop=True)
    cols = instru_count.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    instru_count = instru_count[cols]
    
    x = pd.DataFrame(spreads1['instrument'].unique())
    y = pd.DataFrame(instru_count['instrument'].unique())
    common1 = x.merge(y, on=[0])
    common1 = list(common1[0].values)
    freq = instru_count.loc[instru_count['instrument'].isin(common1)]
    
    for i in range(1, (len(freq['instrument'])+1)):
        freq['k'][i]
        a = freq['k'][i]-1
        for j in range (0, a):
            spreads_unstk = pd.concat([spreads_unstk,spreads_unstk.iloc[:,j]], axis = 1)

    #Sorting the spreads_unstk dataframe by column names
    spreads_unstk = spreads_unstk.sort_index(axis = 1)
    
    #Resetting the index of the dataframe
    spreads_unstk = spreads_unstk.reset_index(drop=True)
    
    #Creating an empty matrix
    spread_extend = np.zeros((n+1, len(tx_bond['instrument'])-len(spreads_unstk.columns)))
    spread_extend = pd.DataFrame(spread_extend)  
    
    #Concatenating spreads_unstk and spread_extend dataframes
    spreads_unstk = pd.concat([spread_extend,spreads_unstk],axis=1)
    
    #Renaming the column names of spreads_unstk by using the column names of interpolated_curve_rate
    spreads_unstk.columns =interpolated_curve_rate.columns
 

In [ ]:
spreads_unstk

In [ ]:
#melt takes related columns with common values and bundles them into one column called ‘variable’. 
spreads_all = spreads_unstk.melt(id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None, ignore_index=True)
spreads_all


In [ ]:
# using melt function on interpolated_curve_rate dataframe
interpolated_curve_rates = interpolated_curve_rate.melt(id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None, ignore_index=True)
interpolated_curve_rates


In [ ]:
##Adding spreads_unstk and interpolated_curve_rate columns
final_ytm = spreads_unstk +interpolated_curve_rate
final_ytm

In [ ]:
#Transposing the final_ytm dataframe
t_final_ytm = final_ytm.T
t_final_ytm

In [ ]:
t_final_ytm = t_final_ytm.eval('instrument = index')
cols1 = t_final_ytm.columns.tolist()
cols1 = cols1[-1:] + cols1[:-1]
t_final_ytm = t_final_ytm[cols1]
t_final_ytm

In [ ]:
# using melt function on t_final_ytm dataframe
t_final_ytm2 = t_final_ytm.melt(id_vars='instrument', value_vars=None, var_name=None, value_name='value', col_level=None, ignore_index=True)
t_final_ytm2

In [ ]:
#To sort the dataframe instrument and variable wise
t_final_ytm2 = t_final_ytm2.sort_values(['instrument', 'variable'], ascending=[True, True])
t_final_ytm2

In [ ]:
#Renaming instrument column as key
t_final_ytm2.rename({'instrument': 'key'}, axis=1, inplace=True)

In [ ]:
 t_final_ytm2

In [ ]:
#Merge two dataframe that is tx_bond_revamp and t_final_ytm2
t_final_ytm_merge = pd.merge(tx_bond_revamp, t_final_ytm2, on='key')

In [ ]:
t_final_ytm_merge

In [ ]:
t_final_ytm_merge['interpolated_curve_rates'] = interpolated_curve_rates['value']
t_final_ytm_merge

In [ ]:
t_final_ytm_merge['value'] = t_final_ytm_merge['value'] + t_final_ytm_merge['mark_up_value']
t_final_ytm_merge

In [ ]:
t_final_ytm_merge['spreads_all'] = spreads_all['value']
t_final_ytm_merge

In [ ]:
##To generate index by considering particular category which has a sequence from 0 to 250 and repeats itself as per the shape of the dataframe

In [ ]:
new_res_df = pd.DataFrame()
for a, g in t_final_ytm_merge.groupby(["key"]):
    new_df = g.reset_index(drop = True)
    new_res_df = new_res_df.append(new_df)

In [ ]:
t_final_ytm_merge_new  = new_res_df.reset_index(drop = False)

In [ ]:
t_final_ytm_merge_new

In [ ]:
t_final_ytm_merge_new.rename(columns = {'index':'env'}, inplace = True)


In [ ]:
t_final_ytm_merge_new

In [ ]:
# Move first column to the Last for better arrangement purpose
temp_cols=t_final_ytm_merge_new.columns.tolist()
new_cols=temp_cols[1:] + temp_cols[0:1]
t_final_ytm_merge_new=t_final_ytm_merge_new[new_cols]
t_final_ytm_merge_new

# All_MTM


In [ ]:
t_final_ytm_merge_new['all_mtm'] =t_final_ytm_merge_new['Cashflow'] / ((1+(t_final_ytm_merge_new['value']/100))**(t_final_ytm_merge_new['residual_maturity_Actual']/t_final_ytm_merge_new['no_of_days_in_a_year']))
t_final_ytm_merge_new

In [ ]:
#Converting MTM dataframe to csv file
t_final_ytm_merge_new.to_csv('Result_mtm.csv')

In [ ]:
##To fetch particular columns from the dataframe and storing it in a dataframe that is 'Result'

In [ ]:
Result = t_final_ytm_merge_new[['env', 'instrument','key','k','Cashflow','Cashflow_Type','Face_value','YTM','cashflow_mtm','value','all_mtm']]

In [ ]:
Result

# CHANGE_IN_MTM


In [ ]:
##Concern: This is a R code where other combinations of functions are used . Here I have simple computed change_in_mtm by subtracting the particular columns 
#result_1 <- ddply(Result, .(key,instrument,env), mutate, change_in_mtm = all_mtm - cashflow_mtm)


In [ ]:
Result['change_in_mtm'] = Result['all_mtm'] - Result['cashflow_mtm']

In [ ]:
Result

In [ ]:
#Generating key2
Result["key2"] = Result['instrument'].map(str) + '_' + Result['env'].map(str) 
Result

# TOTAL CHANGE IN MTM


In [ ]:
#Grouping by two categories that is by variable 'instrument' and 'env' and finding sum over 'change_in_mtm' by considering particular vertices

In [ ]:
total_change_in_MTM = pd.DataFrame(Result.groupby(["instrument", "env"])["change_in_mtm"].sum())

In [ ]:
total_change_in_MTM

In [ ]:
total_change_in_MTM  = total_change_in_MTM.reset_index(drop = False)

In [ ]:
total_change_in_MTM

In [ ]:
total_change_in_MTM = total_change_in_MTM.sort_values(by=['env'])

In [ ]:
total_change_in_MTM.reset_index(drop=True)

In [ ]:
##Sorting the dataframe by instrument and env wise
total_change_in_MTM = total_change_in_MTM.sort_values(by=["instrument","env"])

In [ ]:
total_change_in_MTM

# PORTFOLIO TOTAL CHANGE IN MTM

In [ ]:
#Grouping by env and summimg over change_in_mtm

In [ ]:
portfolio_change_in_MTM = total_change_in_MTM.groupby('env')['change_in_mtm'].sum()

In [ ]:
portfolio_change_in_MTM

In [ ]:
portfolio_change_in_mtm = list(portfolio_change_in_MTM)
portfolio_change_in_mtm.pop(0) ##Removing the first value that is 0
portfolio_change_in_mtm

In [ ]:
len(portfolio_change_in_mtm)

In [ ]:
# Portfolio_change_in_MTM = pd.DataFrame(portfolio_change_in_mtm)
# Portfolio_change_in_MTM = Portfolio_change_in_MTM.sort_values(by = [0],ascending = False)
# Portfolio_change_in_MTM = list(Portfolio_change_in_MTM)

In [ ]:
portfolio_change_in_mtm

# VaR Computations for PORTFOLIO

In [ ]:
hp

# Computing VaR when df = 0 and hp = 1

In [ ]:
# ##Extracting parameters
# decay_factor = hp['decay_factor']
# holding_period = hp['holding_period']
# confidence_level = hp['confidence_level']

<!-- VaR_Exact one day -->

In [ ]:
# VaR_exact_one_day=-Portfolio_change_in_MTM.quantile(1-hp['confidence_level'], axis = 0)
# VaR_exact_one_day.rename(columns = {0:'VaR_exact_one_day'}, inplace = True)
# VaR_exact_one_day

In [ ]:
# ##Using Math.floor function to round off to nearest integer
# nearby = math.floor(confidence_level * len(Portfolio_change_in_MTM))
# nearby_nxt = nearby + 1
# nearby_nxt

In [ ]:
# Portfolio_change_in_MTM.iloc[nearby]

In [ ]:
# Transform as numeric
# PCM = Portfolio_change_in_MTM
# VE = VaR_exact_one_day

In [ ]:
#diff1 = (VE + PCM.iloc[nearby])

In [ ]:
#diff2 = (- PCM.iloc[nearby_nxt] - VE)

In [ ]:
#diff1

In [ ]:
# ###To calculate VaR nearby for one day
# if (diff1 == diff2):
#     VaR_nearby_one_day = - PCM.iloc[nearby + 1]

# elif (diff1 < diff2):
#     VaR_nearby_one_day = - PCM.iloc[nearby]
    
# else:
#     VaR_nearby_one_day = - PCM.iloc[nearby + 1]

In [ ]:
# ##To calculate VaR nearby
# if (diff1 == diff2).bool():
#     VaR_nearby_one_day = - PCM.iloc[nearby + 1]

# elif (diff1 < diff2).bool():
#     VaR_nearby_one_day = - PCM.iloc[nearby]
    
# else:
#     VaR_nearby_one_day = - PCM.iloc[nearby + 1]

In [ ]:
# VaR_nearby_one_day ###here name is not changed instead it is showing the previous variables

In [ ]:
# Portfolio_change_in_MTM

In [ ]:
# portfolio_change_in_MTM

In [ ]:
prob = (1 - confidence_level)
decay_weights = []
    
if (decay_factor==0):
    VaR_exact_one_day = -(np.quantile(portfolio_change_in_mtm, prob))
    Portfolio_change_in_MTM = sorted(portfolio_change_in_mtm, reverse = True)
    nearby = np.floor(confidence_level * len(Portfolio_change_in_MTM))
    nearby = int(nearby)
    nearby_nxt = nearby + 1
    
    diff1 = pd.to_numeric(VaR_exact_one_day + Portfolio_change_in_MTM[nearby])
    diff2 = pd.to_numeric(- Portfolio_change_in_MTM[nearby_nxt] - VaR_exact_one_day)
                          
    if (diff1 == diff2):
        VaR_nearby_one_day = - Portfolio_change_in_MTM[nearby + 1]
    elif (diff1 < diff2):
        VaR_nearby_one_day = - Portfolio_change_in_MTM[nearby]
    else:
        VaR_nearby_one_day = - Portfolio_change_in_MTM[nearby + 1]



else:
    a = []
    a = list((Result['env'].unique()))
    a.pop()
    for number in a:
        decay_weights.append((1-decay_factor)*(decay_factor ** number))
    
    total_mtm = result.groupby('idx')[['all_mtm']].sum
    mtm_returns = []
    
    for i in range(1,(len(change_df['idx'].unique() -1))):
                   fv = total_mtm['mtm_bc'][0]
                   returns = (total_mtm['mtm_bc'][i] - fv)/fv
                   mtm_returns.append(returns)
    
    decay_weights = pd.DataFrame(decay_weights, columns = ['decay_weights'])
    mtm_returns = pd.DataFrame(mtm_returns, columns = ['mtm_returns'])
    mtm = pd.concat([mtm_returns,decay_weights], axis = 1)
    mtm = mtm.sort_values(['mtm_returns'])
    mtm['cum_decay_weights'] = np.cumsum(mtm['decay_weights'])
    a = 1-confidence_level
    mtm1 = mtm.loc[mtm['cum_decay_weights'] < a]
    
    if (len(mtm1['cum_decay_weights'])==0):
                  VaR_exact_one_day = np.abs(- mtm['mtm_returns'][0] * total_mtm['mtm_bc'][0])



                  VaR_exact_one_day = VaR_exact_one_day



    else:
                  index = len(mtm1['cum_decay_weights'])
                                         
                  interpolated_return_exact = mtm['mtm_returns'][index] + ((mtm['mtm_returns'][index + 1] - mtm['mtm_returns'][index]) * ((1 - confidence_level) - mtm['cum_decay_weights'][index]) / (mtm['cum_decay_weights'][index + 1] - mtm['cum_decay_weights'][index] ))
    
                  VaR_exact_one_day =  ((total_mtm[1] * (interpolated_return_exact)))
    
                  if((((1 - confidence_level) - mtm['cum_decay_weights'][index]) < (mtm['cum_decay_weights'][index + 1] - (1 - confidence_level) ))):
            
                            interpolated_return_nearby = mtm['mtm_returns'][index]
                  else:
                            index = len(mtm1['cum_decay_weights'])
                            interpolated_return_nearby = mtm['mtm_returns'][index + 1]
    
                            VaR_nearby_one_day = np.abs(total_mtm['mtm_bc'][0] * (interpolated_return_nearby))

In [ ]:
type(total_)

In [ ]:
VaR_exact_one_day

In [ ]:
VaR_nearby_one_day

In [ ]:
Result

In [ ]:
decay_factor = 0.94

In [ ]:
decay_weights = (1-decay_factor)* ((decay_factor ** range(0,(len(Result['env'].unique())-1)))/((1 - decay_factor) ** (len(Result['env'].unique()) -1)))


In [ ]:
list(0,(len(Result['env'].unique())-1))/((1 - decay_factor) ** (len(Result['env'].unique()) -1))


In [ ]:
a = []
a = list((Result['env'].unique()))
a.pop()
#a = a/(1 - decay_factor) ** 250
a

In [ ]:
decay_weights = []
for number in a:
    decay_weights.append((1-decay_factor)*(decay_factor ** number))

In [ ]:
/ ((1 - decay_factor) ** (len(Result['env'].unique()) -1))))

In [ ]:
len(Result['env'].unique()) -1

In [ ]:
a

In [ ]:
decay_weights

In [ ]:
((1 - decay_factor) ** (len(Result['env'].unique()) -1))

In [ ]:
(1 - decay_factor)**250

In [ ]:
(len(Result['env'].unique())-1)

In [ ]:
a = range(0,(len(Result['env'].unique())-1))
a

In [ ]:
decay_weights = (1-decay_factor) * (hp['decay_factor'].pow(a))/((1 - decay_factor) ** (len(Result['env'].unique() -1)))


# Instrument-Wise VaR Computations

In [ ]:
if decay_factor == 0:
    Bondwise_VAR = total_change_in_MTM.groupby('instrument')
    change_in_MTM_sorted = sorted(total_change_in_MTM, reverse = True)
    VaR_exact_one_day = -(np.quantile(total_change_in_MTM, 0.01))
    nearby = np.floor(confidence_level * len(change_in_MTM_sorted))
    nearby = int(nearby)
    nearby_nxt = nearby + 1
    
    diff1 = pd.to_numeric(VaR_exact_one_day + change_in_MTM_sorted[nearby])
    diff2 = pd.to_numeric(- change_in_MTM_sorted[nearby_nxt] - VaR_exact_one_day)
    
    if (diff1 == diff2):
        VaR_nearby_one_day = - change_in_MTM_sorted[nearby + 1]
    elif (diff1 < diff2):
        VaR_nearby_one_day = - change_in_MTM_sorted[nearby]
    else:
        VaR_nearby_one_day = - change_in_MTM_sorted[nearby + 1]
        
else:
    #decay_weights <- ((1-decay_factor)* c(decay_factor^(0:(length(unique(x$env))-2)))/(1-decay_factor^((length(unique(x$env))-1))))
    
    decay_weights = (1-decay_factor)* ((decay_factor ** range(0,(len(change_df['idx'].unique())-1)))/((1 - decay_factor) ** (len(change_df['idx'].unique() -1))))
    total_mtm = change_df.groupby('idx')[['mtm_bc']].sum
    mtm_returns = []

    for i in range(1,(len(change_df['idx'].unique() -1))):
                   fv = total_mtm['mtm_bc'][0]
                   returns = (total_mtm['mtm_bc'][i] - fv)/fv
                   mtm_returns.append(returns) 
                    
    decay_weights = pd.DataFrame(decay_weights, columns = ['decay_weights'])
    mtm_returns = pd.DataFrame(mtm_returns, columns = ['mtm_returns'])
    mtm = pd.concat([mtm_returns,decay_weights], axis = 1)
    mtm = mtm.sort_values(['mtm_returns'])
    mtm['cum_decay_weights'] = np.cumsum(mtm['decay_weights'])
    a = 1-confidence_level
    mtm1 = mtm.loc[mtm['cum_decay_weights'] < a]
                                         
    if (len(mtm1)==0):
                  VaR_exact_equity_hp = np.abs(- mtm['mtm_returns'][0] * total_mtm['mtm_bc'][0])

                  VaR_nearby_equity_hp = VaR_exact_equity_hp

    else:
                  index = len(mtm1['cum_decay_weights'])
                                         
                  interpolated_return_exact = mtm['mtm_returns'][index] + ((mtm['mtm_returns'][index + 1] - mtm['mtm_returns'][index]) * ((1 - confidence_level) - mtm['cum_decay_weights'][index]) / (mtm['cum_decay_weights'][index + 1] - mtm['cum_decay_weights'][index] ))
    
                  VaR_exact_equity_hp =  ((total_mtm[1] * (interpolated_return_exact)))
    
                  if((((1 - confidence_level) - mtm['cum_decay_weights'][index]) < (mtm['cum_decay_weights'][index + 1] - (1 - confidence_level) ))):
            
                            interpolated_return_nearby = mtm['mtm_returns'][index]
                  else:
                            index = len(mtm1['cum_decay_weights'])
                            interpolated_return_nearby = mtm['mtm_returns'][index + 1]
    
                            VaR_nearby_equity_hp = np.abs(total_mtm['mtm_bc'][0] * (interpolated_return_nearby))
                                         
                                         
    